In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from datetime import datetime

HEADERS = {"User-Agent": "Mozilla/5.0"}


# PHASE 1 — BANKS & MICROFINANCE BANKS

banks = [
    {"lender": "Access Bank", "category": "Bank", "url": "https://www.accessbankplc.com/personal/borrowing/personal-loan"},
    {"lender": "UBA Click Credit", "category": "Bank", "url": "https://www.ubagroup.com/nigeria/personal-banking/loans-payments/click-credit/"},
    {"lender": "First Bank (Loans)", "category": "Bank", "url": "https://www.firstbanknigeria.com/personal/loans/"},
    {"lender": "First Bank (FirstCredit)", "category": "Bank", "url": "https://www.firstbanknigeria.com/personal/loans/firstcredit/"},
    {"lender": "GTBank", "category": "Bank", "url": "https://www.gtbank.com/personal-banking/loans"},
    {"lender": "Stanbic IBTC", "category": "Bank", "url": "https://www.stanbicibtcbank.com/nigeriabank/personal/products-and-services/all-loans"},
    {"lender": "Stanbic IBTC (EZ Cash)", "category": "Bank", "url": "https://www.stanbicibtcbank.com/nigeriabank/personal/products-and-services/all-loans/ez-cash"},
    {"lender": "Wema Bank (Salary Loan)", "category": "Bank", "url": "https://wemabank.com/personal/consumer-loans/salary-based-loan/"},
    {"lender": "Wema Bank (Quick Loan)", "category": "Bank", "url": "https://wemabank.com/personal/consumer-loans/quick-loan/"},
    {"lender": "FCMB FastCash", "category": "Bank", "url": "https://www.fcmb.com/fastcash"},
    {"lender": "Fidelity Bank FlyNow", "category": "Bank", "url": "https://www.fidelitybank.ng/flynow/"},
    {"lender": "Cedar MFB", "category": "Microfinance Bank", "url": "https://cedarmfbank.com.ng/cedar-easy-loan/"}
]


# PHASE 2 — FINTECH, FINTECH BANKS & GOV-BACKED CREDIT

fintechs = [
    {"lender": "Carbon", "category": "Fintech", "url": "https://www.getcarbon.co/loans"},
    {"lender": "Renmoney", "category": "Fintech", "url": "https://renmoney.com/how-to-get-a-personal-loan/"},
    {"lender": "FairMoney", "category": "Fintech", "url": "https://fairmoney.io/products/personal-loans"},
    {"lender": "Credit Direct", "category": "Fintech", "url": "https://www.creditdirect.ng/state-loan"},
    {"lender": "FundQuest", "category": "Fintech", "url": "https://fundquestnigeria.com/5-documents-you-need-to-get-a-loan/"},
    {"lender": "Baobab", "category": "Fintech", "url": "https://baobab.com/ng/loans/"},
    {"lender": "Aella Credit", "category": "Fintech", "url": "https://aellaapp.com/credit"},
    {"lender": "CashX", "category": "Fintech", "url": "https://cashx.ng/legal/terms/"},
    {"lender": "Kuda (Personal Loan)", "category": "Fintech Bank", "url": "https://kuda.com/en-ng/personal-loan/"},
    {"lender": "Kuda (Borrow)", "category": "Fintech Bank", "url": "https://kuda.com/en-ng/borrow/"},
    {"lender": "OKash (via Lendsqr)", "category": "Fintech", "url": "https://lendsqr.com/lenders/okash"},
    {"lender": "YouthCred", "category": "Gov-backed", "url": "https://yid.fmyd.gov.ng/youth-cred/"},
    {"lender": "CrediCorp (CALM Fund)", "category": "Gov-backed", "url": "https://www.credicorp.ng/initiatives/calm-fund"}
]

all_lenders = banks + fintechs


# SCRAPER FUNCTION (TRANSPARENCY-FOCUSED)

def scrape_loan_page(lender, category, url):
    try:
        response = requests.get(url, headers=HEADERS, timeout=20)
        soup = BeautifulSoup(response.text, "html.parser")
        text = soup.get_text(" ", strip=True).lower()

        interest_match = re.search(
            r'(\d+(\.\d+)?\s?%.*?(day|daily|month|monthly|year|annual|annum))',
            text
        )

        return {
            "lender_name": lender,
            "lender_category": category,
            "interest_rate_disclosed": "Yes" if interest_match else "No",
            "interest_rate_text": interest_match.group(0) if interest_match else None,
            "tenure_mentioned": "Yes" if any(x in text for x in ["month", "months", "days", "tenure", "repayment"]) else "No",
            "fees_mentioned": "Yes" if any(x in text for x in ["fee", "charge", "processing"]) else "No",
            "penalty_mentioned": "Yes" if "penalty" in text or "late" in text else "No",
            "salary_required": "Yes" if "salary" in text else "No",
            "collateral_required": "Yes" if "collateral" in text else "No",
            "pricing_requires_application": "Yes" if "apply" in text and not interest_match else "No",
            "source_url": url,
            "scrape_date": datetime.today().strftime("%Y-%m-%d")
        }

    except Exception:
        return {
            "lender_name": lender,
            "lender_category": category,
            "interest_rate_disclosed": "Error",
            "interest_rate_text": None,
            "tenure_mentioned": None,
            "fees_mentioned": None,
            "penalty_mentioned": None,
            "salary_required": None,
            "collateral_required": None,
            "pricing_requires_application": None,
            "source_url": url,
            "scrape_date": datetime.today().strftime("%Y-%m-%d")
        }


# RUN SCRAPER

print(" Scraping Nigerian Personal Loan Market (Banks + Fintechs)...")

records = []

for lender in all_lenders:
    records.append(
        scrape_loan_page(
            lender["lender"],
            lender["category"],
            lender["url"]
        )
    )
    time.sleep(1)


# DATAFRAME & EXPORT

df_loans = pd.DataFrame(records)
df_loans.to_csv("Nigeria_Personal_Loan_Market_Transparency.csv", index=False)

df_loans


🔍 Scraping Nigerian Personal Loan Market (Banks + Fintechs)...


,lender_name,lender_category,interest_rate_disclosed,interest_rate_text,tenure_mentioned,fees_mentioned,penalty_mentioned,salary_required,collateral_required,pricing_requires_application,source_url,scrape_date
0,Access Bank,Bank,Error,None,None,None,None,None,None,None,https://www.accessbankplc.com/personal/borrowi...,2026-02-02
1,UBA Click Credit,Bank,No,None,No,No,No,No,No,No,https://www.ubagroup.com/nigeria/personal-bank...,2026-02-02
2,First Bank (Loans),Bank,Yes,10.3 % with a 4 month,Yes,Yes,No,Yes,No,No,https://www.firstbanknigeria.com/personal/loans/,2026-02-02
3,First Bank (FirstCredit),Bank,Yes,8% flat insurance: 5% per transaction a penal ...,Yes,Yes,Yes,Yes,Yes,No,https://www.firstbanknigeria.com/personal/loan...,2026-02-02
4,GTBank,Bank,No,None,No,No,No,No,No,No,https://www.gtbank.com/personal-banking/loans,2026-02-02
5,Stanbic IBTC,Bank,Yes,25% of your retirement savings account (rsa) b...,Yes,Yes,Yes,Yes,No,No,https://www.stanbicibtcbank.com/nigeriabank/pe...,2026-02-02
6,Stanbic IBTC (EZ Cash),Bank,Yes,4% per month,Yes,Yes,Yes,Yes,Yes,No,https://www.stanbicibtcbank.com/nigeriabank/pe...,2026-02-02
7,Wema Bank (Salary Loan),Bank,No,None,No,No,No,No,No,No,https://wemabank.com/personal/consumer-loans/s...,2026-02-02
8,Wema Bank (Quick Loan),Bank,No,None,No,No,No,No,No,No,https://wemabank.com/personal/consumer-loans/q...,2026-02-02
9,FCMB FastCash,Bank,No,None,No,No,No,No,No,No,https://www.fcmb.com/fastcash,2026-02-02
